In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("electoral2015_tour1.xlsx")

In [3]:
col_title_unicode = data.columns.values.tolist()

### We see that the data is in Unicode format, which will cause some trouble later. We will try to convert it to string.

In [4]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')

In [5]:
col_title = map(unicode_to_string,col_title_unicode)

### We replace the column head by new string-format

In [6]:
data.columns = col_title

In [7]:
data.columns.values.tolist()

["Date de l'export",
 'Code du departement',
 'Libelle du departement',
 'Code de la commune',
 'Libelle de la commune',
 'Inscrits',
 'Abstentions',
 '% Abs/Ins',
 'Votants',
 '% Vot/Ins',
 'Blancs',
 '% Blancs/Ins',
 '% Blancs/Vot',
 'Nuls',
 '% Nuls/Ins',
 '% Nuls/Vot',
 'Exprimes',
 '% Exp/Ins',
 '% Exp/Vot',
 'NListe',
 'Nuance Liste',
 'Libelle Abrege Liste',
 'Libelle Etendu Liste',
 'Nom Tete de Liste',
 'Voix',
 '% Voix/Ins',
 '% Voix/Exp',
 'NListe.1',
 'Nuance Liste.1',
 'Libelle Abrege Liste.1',
 'Libelle Etendu Liste.1',
 'Nom Tete de Liste.1',
 'Voix.1',
 '% Voix/Ins.1',
 '% Voix/Exp.1',
 'NListe.2',
 'Nuance Liste.2',
 'Libelle Abrege Liste.2',
 'Libelle Etendu Liste.2',
 'Nom Tete de Liste.2',
 'Voix.2',
 '% Voix/Ins.2',
 '% Voix/Exp.2',
 'NListe.3',
 'Nuance Liste.3',
 'Libelle Abrege Liste.3',
 'Libelle Etendu Liste.3',
 'Nom Tete de Liste.3',
 'Voix.3',
 '% Voix/Ins.3',
 '% Voix/Exp.3',
 'NListe.4',
 'Nuance Liste.4',
 'Libelle Abrege Liste.4',
 'Libelle Etendu Liste

###  We see that not only the column header but all the text-format cells in the dataset are in unicode. We will now convert all to string.

In [8]:
def data_to_string(data):
    new_data = data.copy()
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
    return new_data

In [9]:
data = data_to_string(data)

### Check if there are NaN values in Numeric columns

In [10]:
def check_nan_in_numeric(data):
    new_data = data.copy()
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) is not str : #check the first not-NaN value to eliminate all string column
            if new_data[col].isnull().any(): 
                print col

In [11]:
check_nan_in_numeric(data)

NListe.8
Voix.8
% Voix/Ins.8
% Voix/Exp.8
NListe.9
Voix.9
% Voix/Ins.9
% Voix/Exp.9
NListe.10
Voix.10
% Voix/Ins.10
% Voix/Exp.10
NListe.11
Voix.11
% Voix/Ins.11
% Voix/Exp.11
NListe.12
Voix.12
% Voix/Ins.12
% Voix/Exp.12


### The problem here is that the number of parties is not the same in all communities: most of them have 8 parties, but some of them have more than 8. 
### Therefore, before checking for NaN value, we need to re-organize the data.
### We will create a new dataframe that has only 1 voting result per row.
### We start by adding a new column in the original dataset: Code Insee. We will use this as our pivot column.

In [12]:
def add_insee_code(data):
    new_data = data.copy()
    insee_code = []
    for x in xrange(len(new_data)):
        if new_data['Code du departement'][x] < 100:
            code = new_data['Code du departement'][x]*1000 + new_data['Code de la commune'][x]
        else:
            code = (new_data['Code du departement'][x]/10)*1000 + new_data['Code de la commune'][x]
        insee_code.append(code)
    new_data['Code Insee'] = insee_code
    cols = new_data.columns.tolist()
    cols = cols[-1:] + cols[:-1] # we move the new variable to the first column of our dataframe
    new_data = new_data[cols]
    return new_data

In [13]:
data = add_insee_code(data)

#### (NB: we had to manually modify some departement codes like those of Corse, Reunion, Martinique,.... because they were not in numerical form).

### Now we create the new dataframe. 

In [14]:
def remove_nan(data):
    new_data = data.copy()
    boolean_df = pd.notnull(new_data[new_data.columns.tolist()[1:]])
    boolean_list = [any(row) for index, row in boolean_df.iterrows()]
    new_data = new_data[boolean_list]
    return new_data

In [15]:
def create_voting_data(data):
    data_index = ['Code Insee', 'NListe', 'Nuance Liste', 'Voix', '% Voix/Ins','% Voix/Exp']

    code_insee = 'Code Insee'
    nListe = 'NListe'
    nuance_liste = 'Nuance Liste'
    voix = 'Voix'
    voix_ins = '% Voix/Ins'
    voix_exp = '% Voix/Exp'

    voting_data = data[data_index] 
    counter = 1

    while True:
        new_nListe = nListe + '.' + str(counter)
        new_nuance_liste = nuance_liste + '.' + str(counter)
        new_voix = voix + '.' + str(counter)
        new_voix_ins = voix_ins + '.' + str(counter)
        new_voix_exp = voix_exp + '.' + str(counter)

        try: # condition to stop
            data[new_nListe] 
        except: 
            break

        new_data_index = ['Code Insee',new_nListe,new_nuance_liste, new_voix,new_voix_ins, new_voix_exp]
        new_data = data[new_data_index]
        new_data.columns = data_index
        voting_data = pd.concat([voting_data, new_data])
        counter += 1

    voting_data = remove_nan(voting_data)
    voting_data = voting_data.sort_index() 
    voting_data.index = range(0,len(voting_data))
    
    return voting_data

In [16]:
voting_data = create_voting_data(data)

In [17]:
voting_data

,Code Insee,NListe,Nuance Liste,Voix,% Voix/Ins,% Voix/Exp
0,1001,5,LEXG,4,0.68,1.25
1,1001,3,LFN,109,18.66,34.06
2,1001,2,LDVD,0,0.00,0.00
3,1001,10,LUG,76,13.01,23.75
4,1001,4,LCOM,8,1.37,2.50
5,1001,9,LDLF,7,1.20,2.19
6,1001,1,LDIV,1,0.17,0.31
7,1001,8,LVEG,15,2.57,4.69
8,1001,6,LUD,100,17.12,31.25
9,1002,6,LUD,27,12.05,21.26


### Once we have a clean list, we will divide it into 3 groups corresponding to the 3 political orienatations: Left, Right and Extreme Right 

In [18]:
list(set(voting_data['Nuance Liste']))

['LDVG',
 'LDVD',
 'LECO',
 'LDLF',
 'LCOM',
 'LUDI',
 'LDIV',
 'LFN',
 'LFG',
 'LEXD',
 'LEXG',
 'LUD',
 'LREG',
 'LUG',
 'LSOC',
 'LVEG',
 'LMDM',
 'LRDG',
 'LLR',
 'LVEC']

In [19]:
left_party = ['LDVG', 'LCOM', 'LFG', 'LEXG','LSOC','LVEG','LREG','LUG']
right_party = ['LDVD','LDLF', 'LUDI','LUD','LMDM', 'LRDG','LLR'] 
er_party = ['LFN','LEXD',] 
other_party = ['LECO','LDIV', 'LVEC']

In [20]:
left_party_vote = voting_data[voting_data['Nuance Liste'].isin(left_party)]
left_party_vote.index = range(0, len(left_party_vote))
                              
right_party_vote = voting_data[voting_data['Nuance Liste'].isin(right_party)]
right_party_vote.index = range(0, len(right_party_vote))
                               
er_party_vote = voting_data[voting_data['Nuance Liste'].isin(er_party)]
er_party_vote.index = range(0, len(er_party_vote))
                            
other_party_vote = voting_data[voting_data['Nuance Liste'].isin(other_party)]
other_party_vote.index = range(0, len(other_party_vote))

### Now we group all the result of the right/left party in each town

In [21]:
def groupby_town(data, original_data):
    new_data = data.copy()
    new_data = new_data.drop(['NListe','Nuance Liste'],1)
    new_data = new_data.groupby('Code Insee')
    new_data = new_data.aggregate(np.sum).reset_index()
    abs_ins = original_data[['Code Insee','% Abs/Ins']] # we add abstention information
    new_data = pd.merge(new_data,abs_ins, on = 'Code Insee',how='left')
    return new_data

In [22]:
left_party_vote = groupby_town(left_party_vote,data) 
right_party_vote = groupby_town(right_party_vote,data)
er_party_vote = groupby_town(er_party_vote,data)
other_party_vote = groupby_town(other_party_vote,data)

In [23]:
right_party_vote.head() 

,Code Insee,Voix,% Voix/Ins,% Voix/Exp,% Abs/Ins
0,1001,107,18.32,33.44,44.69
1,1002,32,14.28,25.20,41.52
2,1004,1096,13.45,29.76,53.34
3,1005,168,14.38,31.22,52.23
4,1006,22,20.56,42.31,49.53


In [24]:
left_party_vote.head()

,Code Insee,Voix,% Voix/Ins,% Voix/Exp,% Abs/Ins
0,1001,103,17.63,32.19,44.69
1,1002,61,27.23,48.04,41.52
2,1004,1345,16.50,36.53,53.34
3,1005,153,13.10,28.44,52.23
4,1006,13,12.15,25.00,49.53


### We export these dataframes to excel files

In [25]:
writer = pd.ExcelWriter('left_party_vote.xlsx')
left_party_vote.to_excel(writer,'Sheet1')
writer.save()

In [26]:
writer = pd.ExcelWriter('right_party_vote.xlsx')
right_party_vote.to_excel(writer,'Sheet1')
writer.save()

In [27]:
writer = pd.ExcelWriter('er_party_vote.xlsx')
er_party_vote.to_excel(writer,'Sheet1')
writer.save()